Para cada querie é preciso mudar ENHANCED, QUERY_NUMBER e QUERY_URL

In [ ]:
# SETUP
import matplotlib.pyplot as plt
from sklearn.metrics import PrecisionRecallDisplay
import numpy as np
import json
import requests
import pandas as pd

ENHANCED = False
QUERY_NUMBER = "q1"
 
QUERY_PATH = '../queries/' + QUERY_NUMBER + '/' + QUERY_NUMBER
QRELS_FILE = QUERY_PATH + '_relevant.txt'
QUERY_URL = ...

if ENHANCED: QUERY_PATH = QUERY_PATH + '_enhanced'

RANKED_DOCUMENTS = QUERY_PATH + '_rank.txt'
RESULTS = QUERY_PATH + '_results.tex'
PRECISION_RECALL = QUERY_PATH + '_pr.pdf'

SYSTEM = 'BASE'
if ENHANCED: SYSTEM = 'ENHANCED'

# Read qrels to extract relevant documents
relevant = list(map(lambda el: el.strip(), open(QRELS_FILE).readlines()))
# Get query results from Solr instance
results = requests.get(QUERY_URL).json()['response']['docs']

ranked_documents = [(index + 1, doc['id']) for index, doc in enumerate(results)]

# Criando um DataFrame a partir da lista de tuplas
df1 = pd.DataFrame(ranked_documents, columns=['RANK', SYSTEM], index=None)

# Convertendo o DataFrame em uma tabela LaTeX sem índice
latex_table = df1.to_latex(index=False)

with open(RANKED_DOCUMENTS,'w') as tf:
    tf.write(latex_table)

print(latex_table)

In [ ]:
# METRICS TABLE
# Define custom decorator to automatically calculate metric based on key
metrics = {}
metric = lambda f: metrics.setdefault(f.__name__, f)

@metric
def ap(results, relevant):
    """Average Precision"""
    precision_values = []
    relevant_count = 0

    for idx, doc in enumerate(results):
        if doc['id'] in relevant:
            relevant_count += 1
            precision_at_k = relevant_count / (idx + 1)
            precision_values.append(precision_at_k)

    if not precision_values:
        return 0.0

    return sum(precision_values)/len(precision_values)

@metric
def p10(results, relevant, n=10):
    """Precision at N"""
    return len([doc for doc in results[:n] if doc['id'] in relevant])/n

def calculate_metric(key, results, relevant):
    return metrics[key](results, relevant)

# Define metrics to be calculated
evaluation_metrics = {
    'ap': 'Average Precision',
    'p10': 'Precision at 10 (P@10)'
}

# Calculate all metrics and export results as LaTeX table
df = pd.DataFrame([['Metric','Value']] +
    [
        [evaluation_metrics[m], calculate_metric(m, results, relevant)]
        for m in evaluation_metrics
    ]
)

print(df)

with open(RESULTS,'w') as tf:
    tf.write(df.to_latex())

In [ ]:
# PRECISION-RECALL CURVE
# Calculate precision and recall values as we move down the ranked list
precision_values = [
    len([
        doc 
        for doc in results[:idx]
        if doc['id'] in relevant
    ]) / idx 
    for idx, _ in enumerate(results, start=1)
]

recall_values = [
    len([
        doc for doc in results[:idx]
        if doc['id'] in relevant
    ]) / len(relevant)
    for idx, _ in enumerate(results, start=1)
]

precision_recall_match = {k: v for k,v in zip(recall_values, precision_values)}

# Extend recall_values to include traditional steps for a better curve (0.1, 0.2 ...)
recall_values.extend([step for step in np.arange(0.1, 1.1, 0.1) if step not in recall_values])
recall_values = sorted(set(recall_values))

# Extend matching dict to include these new intermediate steps
for idx, step in enumerate(recall_values):
    if step not in precision_recall_match:
        if recall_values[idx-1] in precision_recall_match:
            precision_recall_match[step] = precision_recall_match[recall_values[idx-1]]
        else:
            precision_recall_match[step] = precision_recall_match[recall_values[idx+1]]

disp = PrecisionRecallDisplay([precision_recall_match.get(r) for r in recall_values], recall_values)
disp.plot()
plt.savefig(PRECISION_RECALL)
